- https://ddosdb.org/attack-trace/80f37aac99809f9890e3d5dd9993a2a6
- https://ddosdb.org/attack-trace/02f2204ded5031080840041c9d22d5ed
- https://ddosdb.org/attack-trace/f2724bf2d516c91e8373f7153a595ff1_4

In [5]:
import pandas as pd 
import numpy as np

In [21]:
def processing_pcap_ttl(pcap_file):
    !tshark -n -r $pcap_file -E separator=\;  -E header=y -T fields -e ip.src -e ip.ttl  > pcap.txt
    df = pd.read_csv('pcap.txt', error_bad_lines=False, sep=';')
    
    try:
        df['ip.src'] = df['ip.src'].apply(lambda x: x.split(',')[0])
    except:
        pass
    
    try:
        df['ip.ttl'] = df['ip.ttl'].apply(lambda x: int(x.split(',')[0]))
    except:
        pass
    
    df_summary = df.groupby(['ip.src'])['ip.ttl'].agg([np.ptp]).reset_index()
    summary = df_summary['ptp'].value_counts()
    return summary

In [29]:
a = processing_pcap_ttl('../80f37aac99809f9890e3d5dd9993a2a6.pcap') ###EXAMPLE 1
a

0    22
1     9
8     1
Name: ptp, dtype: int64

In [30]:
b = processing_pcap_ttl('../02f2204ded5031080840041c9d22d5ed.pcap') ###EXAMPLE 2
b

0      9098
1        11
2         2
63        1
5         1
4         1
3         1
192       1
Name: ptp, dtype: int64

In [31]:
c = processing_pcap_ttl('../f2724bf2d516c91e8373f7153a595ff1_4.pcap') ###EXAMPLE 3
c

0     1043
1       84
4       66
3       24
2       16
13       1
Name: ptp, dtype: int64

## INVERTING & RENAMING & MERGING THE RESULTS

In [55]:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.transpose.html
transposed_a = a.to_frame().T.reset_index()
transposed_a

,index,0,1,8
0,ptp,22,9,1


In [56]:
transposed_a.at[0, 'index'] = '80f37aac99809f9890e3d5dd9993a2a6'
transposed_a

,index,0,1,8
0,80f37aac99809f9890e3d5dd9993a2a6,22,9,1


#### DOING THE SAME FOR THE OTHERS

In [57]:
transposed_b = b.to_frame().T.reset_index()
transposed_b.at[0, 'index'] = '02f2204ded5031080840041c9d22d5ed'
transposed_b

,index,0,1,2,63,5,4,3,192
0,02f2204ded5031080840041c9d22d5ed,9098,11,2,1,1,1,1,1


In [58]:
transposed_c = c.to_frame().T.reset_index()
transposed_c.at[0, 'index'] = 'f2724bf2d516c91e8373f7153a595ff1_4'
transposed_c

,index,0,1,4,3,2,13
0,f2724bf2d516c91e8373f7153a595ff1_4,1043,84,66,24,16,1


#### MERGING, CONCATTING
https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [78]:
df = pd.concat([transposed_a, transposed_b, transposed_c], sort=False).set_index('index')
df

,0,1,8,2,63,5,4,3,192,13
index,,,,,,,,,,
80f37aac99809f9890e3d5dd9993a2a6,22,9,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
02f2204ded5031080840041c9d22d5ed,9098,11,NaN,2.0,1.0,1.0,1.0,1.0,1.0,NaN
f2724bf2d516c91e8373f7153a595ff1_4,1043,84,NaN,16.0,NaN,NaN,66.0,24.0,NaN,1.0


In [79]:
df = df.reindex(columns=sorted(df.columns))
df

,0,1,2,3,4,5,8,13,63,192
index,,,,,,,,,,
80f37aac99809f9890e3d5dd9993a2a6,22,9,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
02f2204ded5031080840041c9d22d5ed,9098,11,2.0,1.0,1.0,1.0,NaN,NaN,1.0,1.0
f2724bf2d516c91e8373f7153a595ff1_4,1043,84,16.0,24.0,66.0,NaN,NaN,1.0,NaN,NaN


### TOTAL

In [80]:
df['total_ips'] = df.sum(axis=1)

In [81]:
df

,0,1,2,3,4,5,8,13,63,192,total_ips
index,,,,,,,,,,,
80f37aac99809f9890e3d5dd9993a2a6,22,9,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,32.0
02f2204ded5031080840041c9d22d5ed,9098,11,2.0,1.0,1.0,1.0,NaN,NaN,1.0,1.0,9116.0
f2724bf2d516c91e8373f7153a595ff1_4,1043,84,16.0,24.0,66.0,NaN,NaN,1.0,NaN,NaN,1234.0


## PERCENTAGES

In [88]:
pd.options.display.float_format = '{:.2f}%'.format
df = df.div(df['total_ips'], axis=0)*100
df

,0,1,2,3,4,5,8,13,63,192,total_ips
index,,,,,,,,,,,
80f37aac99809f9890e3d5dd9993a2a6,68.75%,28.12%,nan%,nan%,nan%,nan%,3.12%,nan%,nan%,nan%,100.00%
02f2204ded5031080840041c9d22d5ed,99.80%,0.12%,0.02%,0.01%,0.01%,0.01%,nan%,nan%,0.01%,0.01%,100.00%
f2724bf2d516c91e8373f7153a595ff1_4,84.52%,6.81%,1.30%,1.94%,5.35%,nan%,nan%,0.08%,nan%,nan%,100.00%


## REMOVING NAN

In [92]:
df = df.fillna(0)
df

,0,1,2,3,4,5,8,13,63,192,total_ips
index,,,,,,,,,,,
80f37aac99809f9890e3d5dd9993a2a6,68.75%,28.12%,0.00%,0.00%,0.00%,0.00%,3.12%,0.00%,0.00%,0.00%,100.00%
02f2204ded5031080840041c9d22d5ed,99.80%,0.12%,0.02%,0.01%,0.01%,0.01%,0.00%,0.00%,0.01%,0.01%,100.00%
f2724bf2d516c91e8373f7153a595ff1_4,84.52%,6.81%,1.30%,1.94%,5.35%,0.00%,0.00%,0.08%,0.00%,0.00%,100.00%
